In [1]:
import numpy as np 
import faiss  # this will import the faiss library

In [2]:
%load_ext memory_profiler

# Data

In [3]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [4]:
xb.shape

(100000, 64)

In [5]:
%memit

peak memory: 97.65 MiB, increment: 0.15 MiB


# Brute Force Search

In [6]:
#IndexFlatIP, IndexFlatL2 is brute force search, no need to train the index
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
100000


In [7]:
k = 4     # we want to see 4 nearest neighbors

In [8]:
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.1751733 7.207629  7.2511625]
 [0.        6.3235645 6.684581  6.7999454]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.5279865 7.6628466]
 [0.        6.7638035 7.2951202 7.3688145]]


In [9]:
%timeit index.search(xq, k) 
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

6.38 s ± 204 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


In [10]:
%memit

peak memory: 140.42 MiB, increment: 0.02 MiB


# Different indexs

## IndexIVFFlat

In [11]:
nlist = 100 #the number of Voronoi cells 
k = 4
quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist)

print(index.is_trained)
index.train(xb)
print(index.is_trained)
print("Index probe:", index.nprobe)
index.add(xb)                  # add may be a bit slower as well

False
True
Index probe: 1


In [12]:
index.nprobe = 1
%timeit index.search(xq, k)
D, I = index.search(xq, k)     # actual search
print(I[-5:])                  # neighbors of the 5 last queries, result not exactly the same as brute force

288 ms ± 36.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[[ 9900  9309  9810 10048]
 [11055 10895 10812 11321]
 [11353 10164  9787 10719]
 [10571 10664 10632 10203]
 [ 9628  9554  9582 10304]]


In [13]:
index.nprobe = 10              # default nprobe is 1, try a few more
%timeit index.search(xq, k)
D, I = index.search(xq, k)
print(I[-5:])                  # neighbors of the 5 last queries

2.26 s ± 96.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


In [14]:
%memit

peak memory: 134.80 MiB, increment: 0.02 MiB


## IndexIVFPQ - Lower memory Volume

The compression is based on a Product Quantizer, that can be seen as an additional level of quantization, that is applied on sub-vectors of the vectors to encode.

In this case, since the vectors are not stored exactly, the distances that are returned by the search method are also approximations.

In [15]:
nlist = 100
m = 8                             # number of subquantizers
k = 4
quantizer = faiss.IndexFlatL2(d)  # this remains the same
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
                                    # 8 specifies that each sub-vector is encoded as 8 bits
index.train(xb)
index.add(xb)

In [16]:
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)

[[   0   78  608  159]
 [   1 1063  555 1150]
 [   2  304  134  179]
 [   3   64    8   16]
 [   4  288  531  159]]
[[1.6703055 6.173921  6.401592  6.481772 ]
 [1.4011948 5.6117706 6.001244  6.423687 ]
 [1.71779   5.609392  6.13204   6.235721 ]
 [1.7679406 6.6566086 6.7365775 6.9475822]
 [1.473252  5.7753015 6.2890706 6.481528 ]]


In [17]:
index.nprobe = 10              # make comparable with experiment above
%timeit index.search(xq, k)     # actual search
D, I = index.search(xq, k)    
print(I[-5:])

1.42 s ± 41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[[10437  9900  8746  9494]
 [10240 10494 10765 11373]
 [10719 11291 10494 10422]
 [ 9638 10125 10122 10040]
 [10304  9229  9485  9905]]


In [18]:
%memit

peak memory: 154.56 MiB, increment: 0.01 MiB


# Index Factory

In [22]:
index = faiss.index_factory(128, "IVF4096, Flat")

In [23]:
index

<faiss.swigfaiss.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7f2dc7948030> >